# **Sparse Attention & Linear Attention in Transformers**
Both **Sparse Attention** and **Linear Attention** are modifications of the standard **self-attention mechanism** in Transformers, designed to improve efficiency when dealing with **long sequences** (e.g., thousands of tokens).

---

# **1. Standard Self-Attention: Why is it Expensive?**
In standard **self-attention**, each token attends to **all other tokens** in the sequence, leading to **quadratic complexity** \( O(N^2 d) \), where:
- \( N \) = sequence length
- \( d \) = embedding dimension

For large sequences (e.g., **GPT-4 handling thousands of tokens**), this is inefficient in terms of **memory and computation**.

To **reduce complexity**, researchers developed:
1. **Sparse Attention** → Reduces the number of tokens attended to.
2. **Linear Attention** → Reformulates the computation to avoid quadratic complexity.

---

# **2. Sparse Attention**
### **What is Sparse Attention?**
Instead of computing attention for **all pairs of tokens**, Sparse Attention selectively allows tokens to **attend only to a subset** of tokens, reducing computational cost.

### **Mathematical Explanation**
Standard self-attention:
\[
A = \text{softmax} \left( \frac{QK^T}{\sqrt{d_k}} \right) V
\]

Sparse attention **introduces a sparse matrix \( S \) instead of a full softmax mask**:

\[
A = \text{softmax} \left( \frac{QK^T \odot S}{\sqrt{d_k}} \right) V
\]

where \( S \) is a **sparsity mask** that allows **only certain tokens** to interact.

---

### **Types of Sparse Attention**
1. **Fixed-Pattern Sparse Attention**
   - Uses **predefined sparsity patterns** (e.g., blocks, local windows).
   - Example: **Longformer** allows each token to attend to its **local neighborhood** + some **global tokens**.

2. **Learnable Sparse Attention**
   - Learns the best **sparsity pattern** dynamically during training.
   - Example: **Reformer** uses a **clustering-based** technique (LSH Attention).

3. **Strided and Block Sparse Attention**
   - Tokens attend at **regular intervals** (strided) or in **blocks**.
   - Example: **BigBird** introduces a combination of **global, local, and random attention**.

---

### **Advantages of Sparse Attention**
✅ **Reduces Complexity**: **From \( O(N^2) \) to \( O(N \log N) \) or even \( O(N) \)**.  
✅ **Scales to Long Sequences**: Used in models like **Longformer, BigBird, and Reformer**.  
✅ **Efficient Memory Usage**: Only computes attention on **important token interactions**.

### **Disadvantages of Sparse Attention**
❌ **Loss of Global Context**: Since not all tokens attend to each other, some **long-distance dependencies** may be missed.  
❌ **Hard to Optimize**: Finding the **best sparsity pattern** requires additional design choices.

---

# **3. Linear Attention**
### **What is Linear Attention?**
Linear Attention **replaces the softmax operation** in self-attention with a more efficient **kernel-based transformation**, reducing the complexity from **\( O(N^2) \) to \( O(N) \)**.

### **Mathematical Reformulation**
Standard self-attention:
\[
A = \text{softmax} \left( \frac{QK^T}{\sqrt{d_k}} \right) V
\]

Linear attention **rearranges** the operation:
\[
A = \left( \phi(Q) \cdot (\phi(K)^T V) \right)
\]

where **\( \phi(x) \)** is a kernel function (e.g., ReLU or exponential kernel), which enables:
\[
\phi(Q) \cdot \left( \sum \phi(K) \cdot V \right)
\]

This allows the computation to be done in **two steps**:
1. Compute **context vectors** \( C = \sum \phi(K) V \) (**linear time**).
2. Multiply each query \( \phi(Q) \) by \( C \), avoiding \( O(N^2) \) complexity.

---

### **Types of Linear Attention**
1. **Kernel-Based Linear Attention**
   - Uses special **kernel functions** (e.g., **ReLU, Laplace**) to approximate attention.
   - Example: **Performer (Favored Kernel Attention)**.

2. **Low-Rank Linear Attention**
   - Decomposes attention matrices using **low-rank approximations**.
   - Example: **Linformer** reduces the sequence length using a learned projection.

3. **Memory-Based Linear Attention**
   - Stores **compressed memory representations** instead of computing attention at every step.
   - Example: **Synthesizer replaces attention with a learned weight matrix**.

---

### **Advantages of Linear Attention**
✅ **Truly Scales to Large Contexts**: **From \( O(N^2) \) to \( O(N) \)**.  
✅ **Lower Memory Footprint**: Efficient for **long documents (100K+ tokens)**.  
✅ **No Need for Sparsity Masks**: Unlike sparse attention, it retains some global context.

### **Disadvantages of Linear Attention**
❌ **Less Expressive**: Approximations may **lose** some relationships in the sequence.  
❌ **Harder to Train**: Requires careful choice of kernel functions.

---

# **4. Comparison: Sparse vs. Linear Attention**
| Feature            | Sparse Attention | Linear Attention |
|--------------------|----------------|----------------|
| **Computational Complexity** | \( O(N \log N) \) or \( O(N) \) | \( O(N) \) |
| **Memory Usage**  | Lower than standard | Very low |
| **Preserves Global Context?** | Partially | Approximates it |
| **Works Well For** | **Documents, Images** | **Very long sequences (100K+ tokens)** |
| **Example Models** | **BigBird, Longformer, Reformer** | **Performer, Linformer** |

---

# **5. When to Use What?**
- **Sparse Attention**: **When you want to selectively reduce computation while keeping some global context.**
  - Best for **documents (Longformer, BigBird)** or **structured data (Reformer)**.
  
- **Linear Attention**: **When you need extreme scalability for massive contexts**.
  - Best for **long conversations (Performer)** or **scientific literature search (Linformer)**.

🚀 **Both Sparse and Linear Attention are crucial for making transformers efficient for long sequences!**